In [315]:
import pandas as pd
import re
import numpy as np

# Read CSV
df = pd.read_csv("data/data.csv")

# Inspect
len(df)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8984 entries, 0 to 8983
Columns: 364 entries, PUBID to CVC_HS_LEFT_HGA
dtypes: int64(364)
memory usage: 24.9 MB
None


In [316]:
keep_exact = {
    # identifiers / demographics
    "PUBID",
    "KEY!SEX",
    "KEY!BDATE_Y",
    "KEY!RACE_ETHNICITY",
}
keep_prefixes = (
    # Location / treatment
    "CV_MSA",
    "CV_CENSUS_REGION",
    "CV_URBAN-RURAL",

    # Income outcomes
    "YINC-1700",   # wage & salary income
    "YINC-1800",   # estimated wage income

    # Household background
    "CV_HH_SIZE",
    "CV_INCOME_FAMILY",
    "CV_INCOME_GROSS_YR",
    "CV_HH_INCOME_SOURCE",

    # Education (time-varying)
    "CV_HGC_",                # highest grade completed
    "CV_HIGHEST_DEGREE_",     # degree by year
)
keep_education_summary = {
    "CVC_HGC_EVER",
    "CVC_HIGHEST_DEGREE_EVER",
    "CVC_HS_DIPLOMA",
    "CVC_GED",
    "CVC_AA_DEGREE",
    "CVC_BA_DEGREE",
    "CVC_MA_DEGREE",
    "CVC_PHD_DEGREE",
    "CVC_PROF_DEGREE",
    "CVC_HS_LEFT_DATE",
    "CVC_HS_LEFT_HGC",
}
def is_excluded(col):
    return col.startswith("CV_MIGRATE.")
def keep_col(col):
    if col in keep_exact:
        return True
    if col in keep_education_summary:
        return True
    if is_excluded(col):
        return False
    return any(col.startswith(prefix) for prefix in keep_prefixes)
cols_to_keep = [c for c in df.columns if keep_col(c)]
df = df[cols_to_keep]


In [317]:
print(list(df.columns))
len(df)
df.info()

['PUBID', 'YINC-1700', 'YINC-1800', 'KEY!SEX', 'KEY!BDATE_Y', 'CV_CENSUS_REGION', 'CV_INCOME_GROSS_YR', 'CV_HH_INCOME_SOURCE', 'CV_HH_SIZE', 'CV_MSA', 'CV_URBAN-RURAL', 'KEY!RACE_ETHNICITY', 'YINC-1700.1', 'YINC-1800.1', 'CV_CENSUS_REGION.1', 'CV_HGC_9899', 'CV_INCOME_GROSS_YR.1', 'CV_HH_SIZE.1', 'CV_HIGHEST_DEGREE_9899', 'CV_MSA.1', 'CV_URBAN-RURAL.1', 'YINC-1700.2', 'YINC-1800.2', 'CV_CENSUS_REGION.2', 'CV_HGC_9900', 'CV_INCOME_GROSS_YR.2', 'CV_HH_SIZE.2', 'CV_HIGHEST_DEGREE_9900', 'CV_MSA.2', 'CV_URBAN-RURAL.2', 'YINC-1700.3', 'YINC-1800.3', 'CV_CENSUS_REGION.3', 'CV_HGC_0001', 'CV_INCOME_GROSS_YR.3', 'CV_HH_SIZE.3', 'CV_HIGHEST_DEGREE_0001', 'CV_MSA.3', 'CV_URBAN-RURAL.3', 'YINC-1700.4', 'YINC-1800.4', 'CV_CENSUS_REGION.4', 'CV_HGC_0102', 'CV_INCOME_GROSS_YR.4', 'CV_HH_SIZE.4', 'CV_HIGHEST_DEGREE_0102', 'CV_MSA.4', 'CV_URBAN-RURAL.4', 'YINC-1700.5', 'YINC-1800.5', 'CV_CENSUS_REGION.5', 'CV_HGC_0203', 'CV_INCOME_GROSS_YR.5', 'CV_HH_SIZE.5', 'CV_HIGHEST_DEGREE_0203', 'CV_MSA.5', 'CV_

In [318]:

import re

wave_to_year = {
    "": 1997,
    ".1": 1998,
    ".2": 1999,
    ".3": 2000,
    ".4": 2001,
    ".5": 2002,
    ".6": 2003,
    ".7": 2004,
    ".8": 2005,
    ".9": 2006,
    ".10": 2007,
    ".11": 2008,
    ".12": 2009,
    ".13": 2010,
    ".14": 2011,
    ".15": 2013,
    ".16": 2015,
    ".17": 2017,
    ".18": 2019,
    ".19": 2021,
}

static_cols = {
    "PUBID",
    "KEY!SEX",
    "KEY!BDATE_Y",
    "KEY!RACE_ETHNICITY",
    "CV_HH_INCOME_SOURCE",
    "CVC_HGC_EVER",
    "CVC_HIGHEST_DEGREE_EVER",
    "CVC_GED",
    "CVC_HS_DIPLOMA",
    "CVC_AA_DEGREE",
    "CVC_BA_DEGREE",
    "CVC_PROF_DEGREE",
    "CVC_PHD_DEGREE",
    "CVC_MA_DEGREE",
    "CVC_HS_LEFT_DATE",
    "CVC_HS_LEFT_HGC",
}

def rename_with_year(col):
    col = col.strip()

    # 1. Static columns (no time dimension)
    if col in static_cols:
        return col.lower()

    # 2. Already has explicit year (e.g. _1997)
    if re.search(r"_\d{4}$", col):
        return col.lower()

    # 3. Wave-style suffix (.1, .2, .10, etc.)
    match = re.search(r"(\.\d+)$", col)
    suffix = match.group(1) if match else ""

    if suffix in wave_to_year:
        year = wave_to_year[suffix]
        base = col.replace(suffix, "")
        return f"{base.lower()}_{year}"

    # 4. Fallback (should almost never happen)
    return col.lower()

# Apply renaming
df = df.rename(columns={c: rename_with_year(c) for c in df.columns})

# Sanity checks
print("MSA columns:")
print(sorted(c for c in df.columns if c.startswith("cv_msa_")))

print("\nIncome columns (YINC-1700):")
print(sorted(c for c in df.columns if c.startswith("yinc-1700_")))

# Save clean dataset
df.to_csv("data/test.csv", index=False)
len(df)


MSA columns:
['cv_msa_1997', 'cv_msa_1998', 'cv_msa_1999', 'cv_msa_2000', 'cv_msa_2001', 'cv_msa_2002', 'cv_msa_2003', 'cv_msa_2004', 'cv_msa_2005', 'cv_msa_2006', 'cv_msa_2007', 'cv_msa_2008', 'cv_msa_2009', 'cv_msa_2010', 'cv_msa_2011', 'cv_msa_2013', 'cv_msa_2015', 'cv_msa_2017', 'cv_msa_2019', 'cv_msa_2021']

Income columns (YINC-1700):
['yinc-1700_1997', 'yinc-1700_1998', 'yinc-1700_1999', 'yinc-1700_2000', 'yinc-1700_2001', 'yinc-1700_2002', 'yinc-1700_2003', 'yinc-1700_2004', 'yinc-1700_2005', 'yinc-1700_2006', 'yinc-1700_2007', 'yinc-1700_2008', 'yinc-1700_2009', 'yinc-1700_2010', 'yinc-1700_2011', 'yinc-1700_2013', 'yinc-1700_2015', 'yinc-1700_2017', 'yinc-1700_2019', 'yinc-1700_2021']


8984

In [319]:

df = df.rename(columns=lambda c: re.sub(r"^yinc-1700_(\d{4})$", r"yinc_\1", c))
df = df.rename(columns=lambda c: re.sub(r"^yinc-1800_(\d{4})$", r"yinc_est_\1", c))
print(list(df.columns))

['pubid', 'yinc_1997', 'yinc_est_1997', 'key!sex', 'key!bdate_y', 'cv_census_region_1997', 'cv_income_gross_yr_1997', 'cv_hh_income_source', 'cv_hh_size_1997', 'cv_msa_1997', 'cv_urban-rural_1997', 'key!race_ethnicity', 'yinc_1998', 'yinc_est_1998', 'cv_census_region_1998', 'cv_hgc_9899', 'cv_income_gross_yr_1998', 'cv_hh_size_1998', 'cv_highest_degree_9899', 'cv_msa_1998', 'cv_urban-rural_1998', 'yinc_1999', 'yinc_est_1999', 'cv_census_region_1999', 'cv_hgc_9900', 'cv_income_gross_yr_1999', 'cv_hh_size_1999', 'cv_highest_degree_9900', 'cv_msa_1999', 'cv_urban-rural_1999', 'yinc_2000', 'yinc_est_2000', 'cv_census_region_2000', 'cv_hgc_0001', 'cv_income_gross_yr_2000', 'cv_hh_size_2000', 'cv_highest_degree_0001', 'cv_msa_2000', 'cv_urban-rural_2000', 'yinc_2001', 'yinc_est_2001', 'cv_census_region_2001', 'cv_hgc_0102', 'cv_income_gross_yr_2001', 'cv_hh_size_2001', 'cv_highest_degree_0102', 'cv_msa_2001', 'cv_urban-rural_2001', 'yinc_2002', 'yinc_est_2002', 'cv_census_region_2002', 'cv_h

In [320]:
# CHANGE THE YEAR AT THE END OF ACADEMIC SCHOOL YEAR SO INSTEAD OF 0405 IT IS NOW 2005

def rename_school_year(col):
    """
    Convert cv_hgc_0405 -> cv_hgc_2005
    Convert cv_highest_degree_1314 -> cv_highest_degree_2014
    Leave everything else unchanged
    """
    match = re.search(r'_(\d{2})(\d{2})$', col)
    if match:
        start_yy, end_yy = match.groups()
        end_year = int(end_yy)

        # NLSY97 years: 97–99 → 1997–1999, 00–21 → 2000–2021
        if end_year >= 97:
            full_year = 1900 + end_year
        else:
            full_year = 2000 + end_year

        base = col[:match.start()]
        return f"{base}_{full_year}"

    return col


# Apply renaming
df = df.rename(columns={c: rename_school_year(c) for c in df.columns})


In [321]:
print(list(df.columns))
len(df)
df.info()

['pubid', 'yinc_1997', 'yinc_est_1997', 'key!sex', 'key!bdate_y', 'cv_census_region_1997', 'cv_income_gross_yr_1997', 'cv_hh_income_source', 'cv_hh_size_1997', 'cv_msa_1997', 'cv_urban-rural_1997', 'key!race_ethnicity', 'yinc_1998', 'yinc_est_1998', 'cv_census_region_1998', 'cv_hgc_1999', 'cv_income_gross_yr_1998', 'cv_hh_size_1998', 'cv_highest_degree_1999', 'cv_msa_1998', 'cv_urban-rural_1998', 'yinc_1999', 'yinc_est_1999', 'cv_census_region_1999', 'cv_hgc_2000', 'cv_income_gross_yr_1999', 'cv_hh_size_1999', 'cv_highest_degree_2000', 'cv_msa_1999', 'cv_urban-rural_1999', 'yinc_2000', 'yinc_est_2000', 'cv_census_region_2000', 'cv_hgc_2001', 'cv_income_gross_yr_2000', 'cv_hh_size_2000', 'cv_highest_degree_2001', 'cv_msa_2000', 'cv_urban-rural_2000', 'yinc_2001', 'yinc_est_2001', 'cv_census_region_2001', 'cv_hgc_2002', 'cv_income_gross_yr_2001', 'cv_hh_size_2001', 'cv_highest_degree_2002', 'cv_msa_2001', 'cv_urban-rural_2001', 'yinc_2002', 'yinc_est_2002', 'cv_census_region_2002', 'cv_h

In [322]:
yinc_cols = [c for c in df.columns if c.startswith("yinc") or c.startswith("yinc-est")]
print(df[yinc_cols].describe())

len(yinc_cols), yinc_cols[:10]
missing_codes = [-1, -2, -4, -5]

df[yinc_cols] = (
    df[yinc_cols]
    .replace(missing_codes, np.nan)
    .astype("float")
)

print(df[yinc_cols].describe())
print((df[yinc_cols] > 0).sum())


df.to_csv('data/test2.csv', index=False)

          yinc_1997  yinc_est_1997     yinc_1998  yinc_est_1998     yinc_1999  \
count   8984.000000    8984.000000   8984.000000    8984.000000   8984.000000   
mean     268.613980      -3.786955    606.068010      -3.767030   1136.497329   
std      893.981043       1.028711   1705.574951       1.235284   3178.034570   
min       -4.000000      -4.000000     -5.000000      -5.000000     -5.000000   
25%       -4.000000      -4.000000     -4.000000      -4.000000     -4.000000   
50%       -4.000000      -4.000000     -4.000000      -4.000000     -4.000000   
75%      200.000000      -4.000000    400.000000      -4.000000    700.000000   
max    40000.000000       6.000000  14048.000000       4.000000  25249.000000   

       yinc_est_1999     yinc_2000  yinc_est_2000     yinc_2001  \
count    8984.000000   8984.000000    8984.000000   8984.000000   
mean       -3.631122   2063.160619      -3.475846   3414.551202   
std         1.526694   4525.288697       1.790010   6183.097129   
mi

In [323]:
static_cols = [
    "pubid",
    "key!sex",
    "key!race_ethnicity",
    "key!bdate_y",
    "cv_hh_income_source",
    "cvc_hgc_ever",
    "cvc_highest_degree_ever",
    "cvc_ged",
    "cvc_hs_diploma",
    "cvc_aa_degree",
    "cvc_ba_degree",
    "cvc_prof_degree",
    "cvc_phd_degree",
    "cvc_ma_degree",
    "cvc_hs_left_date",
    "cvc_hs_left_hgc",
]

import re

time_cols = [c for c in df.columns if re.search(r"_\d{4}$", c)]
print(time_cols)

['yinc_1997', 'yinc_est_1997', 'cv_census_region_1997', 'cv_income_gross_yr_1997', 'cv_hh_size_1997', 'cv_msa_1997', 'cv_urban-rural_1997', 'yinc_1998', 'yinc_est_1998', 'cv_census_region_1998', 'cv_hgc_1999', 'cv_income_gross_yr_1998', 'cv_hh_size_1998', 'cv_highest_degree_1999', 'cv_msa_1998', 'cv_urban-rural_1998', 'yinc_1999', 'yinc_est_1999', 'cv_census_region_1999', 'cv_hgc_2000', 'cv_income_gross_yr_1999', 'cv_hh_size_1999', 'cv_highest_degree_2000', 'cv_msa_1999', 'cv_urban-rural_1999', 'yinc_2000', 'yinc_est_2000', 'cv_census_region_2000', 'cv_hgc_2001', 'cv_income_gross_yr_2000', 'cv_hh_size_2000', 'cv_highest_degree_2001', 'cv_msa_2000', 'cv_urban-rural_2000', 'yinc_2001', 'yinc_est_2001', 'cv_census_region_2001', 'cv_hgc_2002', 'cv_income_gross_yr_2001', 'cv_hh_size_2001', 'cv_highest_degree_2002', 'cv_msa_2001', 'cv_urban-rural_2001', 'yinc_2002', 'yinc_est_2002', 'cv_census_region_2002', 'cv_hgc_2003', 'cv_income_gross_yr_2002', 'cv_hh_size_2002', 'cv_highest_degree_2003'

In [324]:
len(time_cols), time_cols


(184,
 ['yinc_1997',
  'yinc_est_1997',
  'cv_census_region_1997',
  'cv_income_gross_yr_1997',
  'cv_hh_size_1997',
  'cv_msa_1997',
  'cv_urban-rural_1997',
  'yinc_1998',
  'yinc_est_1998',
  'cv_census_region_1998',
  'cv_hgc_1999',
  'cv_income_gross_yr_1998',
  'cv_hh_size_1998',
  'cv_highest_degree_1999',
  'cv_msa_1998',
  'cv_urban-rural_1998',
  'yinc_1999',
  'yinc_est_1999',
  'cv_census_region_1999',
  'cv_hgc_2000',
  'cv_income_gross_yr_1999',
  'cv_hh_size_1999',
  'cv_highest_degree_2000',
  'cv_msa_1999',
  'cv_urban-rural_1999',
  'yinc_2000',
  'yinc_est_2000',
  'cv_census_region_2000',
  'cv_hgc_2001',
  'cv_income_gross_yr_2000',
  'cv_hh_size_2000',
  'cv_highest_degree_2001',
  'cv_msa_2000',
  'cv_urban-rural_2000',
  'yinc_2001',
  'yinc_est_2001',
  'cv_census_region_2001',
  'cv_hgc_2002',
  'cv_income_gross_yr_2001',
  'cv_hh_size_2001',
  'cv_highest_degree_2002',
  'cv_msa_2001',
  'cv_urban-rural_2001',
  'yinc_2002',
  'yinc_est_2002',
  'cv_census_re

In [325]:
long_df = df.melt(
    id_vars=static_cols,
    value_vars=time_cols,
    var_name="variable",
    value_name="value"
)

# extract the year (YYYY) from the column name
long_df["year"] = (
    long_df["variable"]
    .str.extract(r"(\d{4})")
    .astype(int)
)
print(long_df['year'].unique())

# remove the _YYYY suffix from the variable name
long_df["variable"] = (
    long_df["variable"]
    .str.replace(r"_\d{4}$", "", regex=True)
)


sorted(long_df["variable"].unique())


[1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
 2011 2012 2013 2014 2015 2017 2019 2021]


['cv_census_region',
 'cv_hgc',
 'cv_hgc_ever_edt',
 'cv_hh_size',
 'cv_highest_degree',
 'cv_highest_degree_ever_edt',
 'cv_income_family',
 'cv_income_gross_yr',
 'cv_msa',
 'cv_urban-rural',
 'yinc',
 'yinc_est']

In [326]:
long_df["year"].value_counts().sort_index().head(10)

#check variable names (should NOT contain years anymore)
long_df["variable"].value_counts().head(10)

# confirm no leftover _YYYY
long_df["variable"].str.contains(r"\d{4}").sum()

0

In [327]:
long_df.columns

Index(['pubid', 'key!sex', 'key!race_ethnicity', 'key!bdate_y',
       'cv_hh_income_source', 'cvc_hgc_ever', 'cvc_highest_degree_ever',
       'cvc_ged', 'cvc_hs_diploma', 'cvc_aa_degree', 'cvc_ba_degree',
       'cvc_prof_degree', 'cvc_phd_degree', 'cvc_ma_degree',
       'cvc_hs_left_date', 'cvc_hs_left_hgc', 'variable', 'value', 'year'],
      dtype='object')

In [328]:
long_df["variable"] = long_df["variable"].replace({
    "yinc": "income_wages",
    "yinc_est": "income_wages_est"
})


In [329]:
panel_df = (
    long_df
    .pivot_table(
        index=static_cols + ["year"],
        columns="variable",
        values="value",
        aggfunc="first"   # critical: do NOT aggregate income
    )
    .reset_index()
)

panel_df.columns.name = None
panel_df.columns = panel_df.columns.str.lower()


In [330]:
target_cols = [
    "income_wages",
    "income_wages_est",
    "hh_income",
    "hh_income_family",
    "highest_grade_completed",
    "highest_degree"
]

for col in target_cols:
    if col in panel_df.columns:
        panel_df.loc[panel_df[col] < 0, col] = np.nan

In [331]:
panel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197648 entries, 0 to 197647
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   pubid                       197648 non-null  int64  
 1   key!sex                     197648 non-null  int64  
 2   key!race_ethnicity          197648 non-null  int64  
 3   key!bdate_y                 197648 non-null  int64  
 4   cv_hh_income_source         197648 non-null  int64  
 5   cvc_hgc_ever                197648 non-null  int64  
 6   cvc_highest_degree_ever     197648 non-null  int64  
 7   cvc_ged                     197648 non-null  int64  
 8   cvc_hs_diploma              197648 non-null  int64  
 9   cvc_aa_degree               197648 non-null  int64  
 10  cvc_ba_degree               197648 non-null  int64  
 11  cvc_prof_degree             197648 non-null  int64  
 12  cvc_phd_degree              197648 non-null  int64  
 13  cvc_ma_degree 

In [332]:
panel_df["age"] = panel_df["year"] - panel_df["key!bdate_y"]
panel_df["teen"] = ((panel_df["age"] >= 13) & (panel_df["age"] <= 18)).astype(int)
panel_df.groupby("pubid")["year"].nunique().describe()
panel_df["income_wages"].notna().sum()
panel_df.to_csv("data/nlsy97_panel.csv", index=False)

In [333]:
panel_df.info()
panel_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197648 entries, 0 to 197647
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   pubid                       197648 non-null  int64  
 1   key!sex                     197648 non-null  int64  
 2   key!race_ethnicity          197648 non-null  int64  
 3   key!bdate_y                 197648 non-null  int64  
 4   cv_hh_income_source         197648 non-null  int64  
 5   cvc_hgc_ever                197648 non-null  int64  
 6   cvc_highest_degree_ever     197648 non-null  int64  
 7   cvc_ged                     197648 non-null  int64  
 8   cvc_hs_diploma              197648 non-null  int64  
 9   cvc_aa_degree               197648 non-null  int64  
 10  cvc_ba_degree               197648 non-null  int64  
 11  cvc_prof_degree             197648 non-null  int64  
 12  cvc_phd_degree              197648 non-null  int64  
 13  cvc_ma_degree 

,pubid,key!sex,key!race_ethnicity,key!bdate_y,cv_hh_income_source,cvc_hgc_ever,cvc_highest_degree_ever,cvc_ged,cvc_hs_diploma,cvc_aa_degree,...,cv_highest_degree,cv_highest_degree_ever_edt,cv_income_family,cv_income_gross_yr,cv_msa,cv_urban-rural,income_wages,income_wages_est,age,teen
count,197648.000000,197648.000000,197648.000000,197648.000000,197648.000000,197648.000000,197648.000000,197648.000000,197648.000000,197648.000000,...,134760.000000,62888.000000,116792.000000,62888.000000,179680.000000,179680.000000,94714.000000,16310.000000,197648.000000,197648.000000
mean,4504.301759,1.488090,2.787845,1982.010352,1.020370,13.722952,2.459595,33.213491,188.418856,42.173642,...,0.646705,0.861691,47318.870539,24497.348524,1.173787,-0.112378,26522.696117,2.176824,25.762376,0.182289
std,2602.997575,0.499859,1.313575,1.396796,0.141261,5.029926,1.557929,99.354307,111.986001,120.790558,...,2.655257,3.322498,66758.757714,44256.838295,2.739134,2.174003,32622.867159,1.161971,6.937355,0.386083
min,1.000000,1.000000,1.000000,1980.000000,1.000000,-3.000000,-3.000000,-4.000000,-4.000000,-4.000000,...,-5.000000,-5.000000,-5.000000,-48100.000000,-5.000000,-5.000000,0.000000,1.000000,13.000000,0.000000
25%,2248.750000,1.000000,1.000000,1981.000000,1.000000,12.000000,2.000000,-4.000000,209.750000,-4.000000,...,0.000000,0.000000,-3.000000,-4.000000,2.000000,0.000000,5000.000000,1.000000,20.000000,0.000000
50%,4501.500000,1.000000,4.000000,1982.000000,1.000000,13.000000,2.000000,-4.000000,245.000000,-4.000000,...,2.000000,2.000000,27000.000000,-2.000000,2.000000,1.000000,18900.000000,2.000000,25.000000,0.000000
75%,6758.250000,2.000000,4.000000,1983.000000,1.000000,16.000000,4.000000,-4.000000,258.000000,-4.000000,...,2.000000,3.000000,70000.000000,36000.000000,3.000000,1.000000,36000.000000,3.000000,31.000000,0.000000
max,9022.000000,2.000000,4.000000,1984.000000,2.000000,95.000000,7.000000,503.000000,502.000000,504.000000,...,7.000000,7.000000,599728.000000,425586.000000,5.000000,2.000000,380288.000000,7.000000,41.000000,1.000000


In [334]:
import numpy as np

num_cols = panel_df.select_dtypes(include=["int64", "float64"]).columns
panel_df[num_cols] = panel_df[num_cols].mask(panel_df[num_cols] < 0)

In [335]:
panel_df["race_black"] = (panel_df["key!race_ethnicity"] == 1).astype(int)
panel_df["race_hispanic"] = (panel_df["key!race_ethnicity"] == 2).astype(int)
panel_df["race_mixed"] = (panel_df["key!race_ethnicity"] == 3).astype(int)

panel_df = panel_df.drop(columns=["key!race_ethnicity"])

In [336]:
panel_df = panel_df.sort_values(["pubid", "year"])
panel_df["moved"] = (
    panel_df.groupby("pubid")["cv_census_region"]
    .diff()
    .notna()
    .astype(int)
)
panel_df["teen_move"] = panel_df["moved"] * panel_df["teen"]


In [337]:
panel_df.to_csv('data/panel_data.csv', index=False)

In [338]:
panel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197648 entries, 0 to 197647
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   pubid                       197648 non-null  int64  
 1   key!sex                     197648 non-null  int64  
 2   key!bdate_y                 197648 non-null  int64  
 3   cv_hh_income_source         197648 non-null  int64  
 4   cvc_hgc_ever                195448 non-null  float64
 5   cvc_highest_degree_ever     196944 non-null  float64
 6   cvc_ged                     25168 non-null   float64
 7   cvc_hs_diploma              148588 non-null  float64
 8   cvc_aa_degree               26070 non-null   float64
 9   cvc_ba_degree               52536 non-null   float64
 10  cvc_prof_degree             2816 non-null    float64
 11  cvc_phd_degree              1430 non-null    float64
 12  cvc_ma_degree               15774 non-null   float64
 13  cvc_hs_left_da